In [1]:
import numpy as np

data = [
    [0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1],
    [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0], 
    [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1], 
    [1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0],  
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1],
    [1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
    [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
    [0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1]
]


In [11]:
import numpy as np

class ART_1:
    def __init__(self, input_size, init_n_classes, rho):
        self.n = input_size
        self.m = init_n_classes
        self.rho = rho     

    def param_init(self, n, m):
        # 初始化 f, b weights & active tag
        self.Wf = np.ones((m, n)) / (1 + n)
        self.Wb = np.ones((n, m))              
        self.act_tag = np.ones(m, dtype=bool)  

    def recognition(self, x):
        # 計算 activation value
        u = np.dot(self.Wf, x)
        # 將 dead neuron 設負無限大
        u[~self.act_tag] = -np.inf  
        k = np.argmax(u)
        return k
    
    def comparison(self, x, k):
        # 計算相似度
        R = np.sum(np.dot(self.Wb[:, k], x)) / np.sum(x)
        return True if R >= self.rho else False
   
    def add_new_neuron(self, x):
        # 在 output layer 新增一個 neuron (多一類別)   
        new_start_idx = self.m
        self.m += 1
        self.act_tag = np.append(self.act_tag, True)
        # Wf, Wb 都需要新增 (多兩條邊)
        self.Wf = np.vstack([self.Wf, np.ones((1, self.n)) / (1 + self.n)])
        self.Wb = np.hstack([self.Wb, np.ones((self.n, 1))])
        # 更新新 neuron 的 weights
        self.update_weights(x, new_start_idx)
        print("(Add)")

    def update_weights(self, x, k):
        # 更新 weights
        print("k_final =", k, end=",  ")
        self.Wf[k, :] = self.Wb[:, k] * x / (0.5 + np.sum(self.Wb[:, k] * x))
        self.Wb[:, k] = self.Wb[:, k] * x
        print("output vector: ", self.provision(x), end="  ")

    def provision(self, x):
        u = np.dot(self.Wf, x) 
        k = np.argmax(u)
        output = np.zeros(self.m, dtype=int)
        output[k] = 1
        return output
    
    def train(self, inputs):
        print(f"rho = {self.rho}, n F2 neuron = {self.m}", end="\n\n")
        self.param_init(self.n, self.m)
        for idx, x in enumerate(inputs):
            print("idx =", idx, end=",  ")
            # 每跑完一個 sample 就重製 active tag
            self.act_tag[:, None] = True
            while True:
                k = self.recognition(x)        
                if self.comparison(x, k):
                    # 若相似度大於 rho 直接更新 weights
                    self.update_weights(x, k)
                    print("(Update)")
                    break
                else:
                    # 若相似度小於 rho 則將 winner neuron 設為 dead
                    self.act_tag[k] = False
                    # 若所有 neuron 都 dead 則新增一個 neuron
                    if not np.any(self.act_tag):
                        self.add_new_neuron(x)
                        break

    def show_class_group(self, inputs):
        group_dict = {}
        for idx in range(self.m):
            group_dict[f"Class-{idx}"] = []
        for idx, x in enumerate(inputs):
            u = np.dot(self.Wf, x)
            k = np.argmax(u)
            group_dict[f"Class-{k}"].append(idx)
        for k, v in group_dict.items():
            print(f"{k}:", v)



In [51]:

art1 = ART_1(16, 1, 0.9)

train_data = np.array(data)

art1.train(train_data)
print("")
art1.show_class_group(train_data)

rho = 0.9, n F2 neuron = 1

idx = 0,  k_final = 0,  output vector:  [1]  (Update)
idx = 1,  k_final = 1,  output vector:  [0 1]  (Add)
idx = 2,  k_final = 2,  output vector:  [0 0 1]  (Add)
idx = 3,  k_final = 3,  output vector:  [0 0 0 1]  (Add)
idx = 4,  k_final = 4,  output vector:  [0 0 0 0 1]  (Add)
idx = 5,  k_final = 0,  output vector:  [1 0 0 0 0]  (Update)
idx = 6,  k_final = 5,  output vector:  [0 0 0 0 0 1]  (Add)
idx = 7,  k_final = 2,  output vector:  [0 0 1 0 0 0]  (Update)
idx = 8,  k_final = 6,  output vector:  [0 0 0 0 0 0 1]  (Add)
idx = 9,  k_final = 4,  output vector:  [0 0 0 0 1 0 0]  (Update)

Class-0: [0, 5]
Class-1: [1]
Class-2: [2, 7]
Class-3: [3]
Class-4: [4, 9]
Class-5: [6]
Class-6: [8]
